Set Colab to GPU Mode if you are training the models!

### Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

/content/drive/.shortcut-targets-by-id/1h_oF16bG32l75GPobgHe_9HgdUduLFxT/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation


### Importing Necessary Libraries


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS





In [4]:
file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [5]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [6]:
data_ori = list()
for file_name in file_names:
  data_ori.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

Now, here we prepare the data to be fed to the Neural Network Architechtures. We can't predict the time ahead upto which the subjects are predicting correctly so, we will take a bit round about way to predict the BOLD values ahead in time(for several choices of time points e.g. 1 time point ahead, 2 time point ahead, etc) and will compare the results for young and old subjects.

Right from the starting we decide some of the data preparation parameters, so that we can use them as we require.

In [7]:
TRAIN_BATCH_SIZE = 20
TEST_BATCH_SIZE = 10
LOOK_AHEAD = 1
NUM_TEST_SUBS = 10

We will start a basic data preparation where we will average across all the voxels for a ROI and fit the model which predicts this average BOLD value. Later we will avoid doing an average since, all voxels in a ROI is not equally important and hence we will want to predict the BOLD values of several voxels simultaneously.

In [8]:
data = [np.mean(dat, axis=1, keepdims=True) for dat in data_ori]

Let's Start the Data Preparation for lOFC later we will iterate the same procedure for the other ROIs.

In [9]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (189, 1, 111)


Below, we see the average BOLD time series plot for all the ROIs.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [10]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [11]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (111, 189, 1)


We split the dataset of 111 young subjects into 101 subjects in the training data and 10 subject in the test data.

In [12]:
train_data_unscaled = [dat[:-NUM_TEST_SUBS] for dat in data]
test_data_unscaled = [dat[-NUM_TEST_SUBS:] for dat in data]  

We scaled the BOLD values of each voxel in the train data between 0 and 1 for each voxel. We use the same MinMaxScaler fit on the train data on the test data.

In [13]:
scalers = list()
for i in range(len(ROI_names)):
  scalers.append(MinMaxScaler(feature_range = (0, 1)))
train_data = [sc.fit_transform(dat[:,:,0]) for sc, dat in zip(scalers, train_data_unscaled)] 
test_data = [sc.transform(dat[:,:,0]) for sc, dat in zip(scalers, test_data_unscaled)]

In [14]:
train_data = [np.expand_dims(dat, axis = -1) for dat in train_data]
test_data = [np.expand_dims(dat, axis = -1) for dat in test_data]

In [15]:
print(ROI_names[-1], "Train Data Shape: ", train_data[-1].shape)
print(ROI_names[-1], "Test Data Shape: ", test_data[-1].shape)

lOFC Train Data Shape:  (101, 189, 1)
lOFC Test Data Shape:  (10, 189, 1)


We now build a data generator which would produce mini-batches during the training of the neural network.

In [16]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_TRs, batch_size, look_ahead=1):
        self.data = data
        self.num_TRs = num_TRs
        self.batch_size = batch_size
        self.current_idx = 0
        self.look_ahead = look_ahead

    def generate(self):
        while True:
            if self.current_idx + self.batch_size >= self.data.shape[0]:
              self.current_idx = 0
            x = self.data[self.current_idx:(self.current_idx+self.batch_size),:(-self.look_ahead),:]
            y = self.data[self.current_idx:(self.current_idx+self.batch_size),self.look_ahead:,:]
            self.current_idx = self.current_idx + self.batch_size
            yield x, y 

In [17]:
train_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TRAIN_BATCH_SIZE, LOOK_AHEAD) for dat in train_data]
test_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TEST_BATCH_SIZE, LOOK_AHEAD) for dat in test_data]

In [18]:
batch_x, batch_y = next(train_data_generator[-1].generate())
print(ROI_names[-1], "Train Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Train Batch y Shape: ", batch_y.shape)

lOFC Train Batch X Shape:  (20, 188, 1)
lOFC Train Batch y Shape:  (20, 188, 1)


In [19]:
batch_x, batch_y = next(test_data_generator[-1].generate())
print(ROI_names[-1], "Test Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Test Batch y Shape: ", batch_y.shape)

lOFC Test Batch X Shape:  (10, 188, 1)
lOFC Test Batch y Shape:  (10, 188, 1)


### Building Model

#### Model-1

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 8`
- `dropout = 0.3`

In [20]:
HIDDEN_UNITS = 8
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [21]:
def avg_bold_model():
  model = Sequential()
  for _ in range(LSTM_LAYERS):
    model.add(LSTM(HIDDEN_UNITS, return_sequences=True))
  model.add(Dropout(0.3))
  for _ in range(DENSE_LAYERS-1):
    model.add(TimeDistributed(Dense(HIDDEN_UNITS)))
  model.add(TimeDistributed(Dense(1)))
  return model

In [22]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (10, 188, 8)              320       
                                                                 
 dropout (Dropout)           (10, 188, 8)              0         
                                                                 
 time_distributed (TimeDistr  (10, 188, 1)             9         
 ibuted)                                                         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [23]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [24]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [25]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [27]:
a_TRs = np.array([56, 67, 73, 81, 134, 142, 155, 167, 174, 180])
d_TRs = np.array([67, 72, 85, 89, 145, 154, 168, 174, 180, 188])
and_TRs = np.union1d(a_TRs, d_TRs)

In [31]:
print("***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
print()
avg_r = list()
for i in range(len(ROI_names)):
  actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
  pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
  for j in range(NUM_TEST_SUBS):
    print(f"{ROI_names[i]} for {j+1}th sub: {np.round(np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1], 2)}", end=" ---> ")
    sum_as = 0 # stores the sum of each the subject's avg BOLD 5 ascents correlation between actual and preds 
    sum_des = 0 # stores the sum of each the subject's avg BOLD 5 descents correlation between actual and preds 
    for k in range(5):
      a = np.round(np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1], 2)
      d = np.round(np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1], 2)
      print(f"A{k+1}: {a}", end=", ")
      sum_as += a
      print(f"D{k+1}: {d}", end=", ")
      sum_des += d
    print(f"Avg. Ascent: {np.round(sum_as/5, 2)}", end=", ")
    print(f"Avg. Descent: {np.round(sum_des/5, 2)}", end="")
    print()
  print()

print("***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
print()
for i in range(len(ROI_names)):
  actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
  pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
  sum_r = 0 # stores the sum of all the subjects avg BOLD correlation between actual and preds
  sum_a = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 ascents correlation between actual and preds
  sum_d = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 descents between actual and preds
  for j in range(NUM_TEST_SUBS):
    sum_r += np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1]
    for k in range(5):
      sum_a[k] += np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1]
      sum_d[k] += np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1]
  print(f"{ROI_names[i]}: {np.round(sum_r / NUM_TEST_SUBS, 2)}", end=", ")
  print(f"Ascents: {np.round(sum_a / NUM_TEST_SUBS, 2)}", end=", ")
  print(f"Avg. Ascent: {np.round(np.mean(sum_a / NUM_TEST_SUBS), 2)}", end=", ")
  print(f"Descents: {np.round(sum_d / NUM_TEST_SUBS, 2)}", end=", ")
  print(f"Avg. Descent: {np.round(np.mean(sum_d / NUM_TEST_SUBS), 2)}")

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***

dmPFC for 1th sub: 0.16 ---> A1: 0.1, D1: 0.04, A2: -0.14, D2: 0.43, A3: 0.44, D3: 0.12, A4: -0.13, D4: -0.1, A5: 0.03, D5: 0.59, Avg. Ascent: 0.06, Avg. Descent: 0.22
dmPFC for 2th sub: 0.34 ---> A1: 0.62, D1: -0.58, A2: -0.38, D2: -0.46, A3: -0.46, D3: 0.05, A4: 0.25, D4: 0.66, A5: -0.18, D5: 0.26, Avg. Ascent: -0.03, Avg. Descent: -0.01
dmPFC for 3th sub: 0.36 ---> A1: 0.43, D1: -0.5, A2: -0.63, D2: -0.66, A3: -0.82, D3: -0.54, A4: -0.11, D4: -0.69, A5: -0.38, D5: 0.02, Avg. Ascent: -0.3, Avg. Descent: -0.47
dmPFC for 4th sub: 0.2 ---> A1: -0.72, D1: -0.28, A2: -0.25, D2: -0.68, A3: -0.48, D3: -0.38, A4: 0.03, D4: 0.48, A5: -0.23, D5: -0.4, Avg. Ascent: -0.33, Avg. Descent: -0.25
dmPFC for 5th sub: 0.21 ---> A1: -0.33, D1: 0.11, A2: -0.65, D2: 0.67, A3: -0.39, D3: 0.22, A4: 0.35, D4: -0.12, A5: -0.3, D5: -0.84, Avg. Ascent: -0.26, Avg. Descent: 0.01